In [1]:
import numpy as np
import itertools
import json

from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve, _SigmoidCalibration
from ampligraph.evaluation import evaluate_performance, mr_score, mrr_score, hits_at_n_score, generate_corruptions_for_eval
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score
from scipy.special import expit

from ampligraph.datasets import load_fb13
from ampligraph.latent_features.models import TransE, ComplEx, DistMult, ConvKB, HolE
import types

In [2]:
from generate_corruptions import generate_corruptions, calibration_loss, pos_iso

In [3]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [4]:
X = load_fb13()

In [5]:
X_valid_pos = X['valid'][X['valid_labels']]
X_valid_neg = X['valid'][~X['valid_labels']]

X_test_pos = X['test'][X['test_labels']]
X_test_neg = X['test'][~X['test_labels']]

In [6]:
losses =  ['self_adversarial', 'pairwise', 'nll', 'multiclass_nll']
models = [TransE, DistMult, ComplEx, HolE]


results = []

for m, l in itertools.product(models, losses):
    model = m(batches_count=32, seed=0, epochs=1000, k=100, eta=20,
               optimizer='adam', optimizer_params={'lr':0.0001},
               loss=l, verbose=False)
    
    try:
        model.fit(X['train'])

        scores = model.predict(X['test'])

        model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
        print("pos", model.calibration_parameters)
        probas1 = model.predict_proba(X['test'])

        model.calibrate(X_valid_pos, X_valid_neg)
        print("pos neg", model.calibration_parameters)
        probas2 = model.predict_proba(X['test'])

        val_scores = model.predict(X['valid'])
        ir = IsotonicRegression(out_of_bounds='clip')
        ir.fit(np.squeeze(val_scores).astype(float), (X['valid_labels']).astype(float))
        probas3 = ir.predict(np.squeeze(scores).astype(float))

        model.generate_corruptions = types.MethodType(generate_corruptions, model)
        corruptions = model.generate_corruptions(X_valid_pos, batches_count=10, epochs=1000)
        val_pos_scores = np.squeeze(model.predict(X_valid_pos))
        iso_pos = pos_iso(IsotonicRegression(out_of_bounds='clip'), val_pos_scores, corruptions, positive_base_rate=0.5)
        probas4 = iso_pos.predict(np.squeeze(scores).astype(float))

        sc_pos = pos_iso(_SigmoidCalibration(), val_pos_scores, corruptions, positive_base_rate=0.5)
        print("pos sc", sc_pos.a_, sc_pos.b_)
        probas5 = sc_pos.predict(np.squeeze(scores).astype(float))

        val_neg_scores = np.squeeze(model.predict(X_valid_neg))
        sc_pos_neg = pos_iso(_SigmoidCalibration(), val_pos_scores, val_neg_scores, positive_base_rate=0.5)
        print("pos neg sc", sc_pos_neg.a_, sc_pos_neg.b_)
        probas6 = sc_pos_neg.predict(np.squeeze(scores).astype(float))

        thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
        thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
        per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)
        
        acc_uncalib = accuracy_score(X['test_labels'], expit(scores) > 0.5)

        acc1 = accuracy_score(X['test_labels'], probas1 > 0.5)
        acc2 = accuracy_score(X['test_labels'], probas2 > 0.5)
        acc3 = accuracy_score(X['test_labels'], probas3 > 0.5)
        acc4 = accuracy_score(X['test_labels'], probas4 > 0.5)
        acc5 = accuracy_score(X['test_labels'], probas5 > 0.5)
        acc6 = accuracy_score(X['test_labels'], probas6 > 0.5)
        
        filter_triples = np.concatenate((X['train'], X_valid_pos, X_test_pos))
        ranks = evaluate_performance(X_test_pos, 
                                     model=model, 
                                     filter_triples=filter_triples,
                                     use_default_protocol=True, 
                                     verbose=False)
    except Exception as e:
        print("Exception: {}".format(e))
        continue
        
    results.append({
        'model': m.__name__,
        'loss': l,
        'brier_score_scores': brier_score_loss(X['test_labels'], expit(scores)),
        'log_loss_scores': log_loss(X['test_labels'], expit(scores), eps=1e-7),
        'brier_score_probas_pos': brier_score_loss(X['test_labels'], probas1),
        'log_loss_probas_pos': log_loss(X['test_labels'], probas1, eps=1e-7),
        'brier_score_probas_pos_neg': brier_score_loss(X['test_labels'], probas2),
        'log_loss_probas_pos_neg': log_loss(X['test_labels'], probas2, eps=1e-7),
        'brier_score_probas_pos_neg_iso': brier_score_loss(X['test_labels'], probas3),
        'log_loss_probas_pos_neg_iso': log_loss(X['test_labels'], probas3, eps=1e-7),
        'brier_score_probas_pos_iso': brier_score_loss(X['test_labels'], probas4),
        'log_loss_probas_pos_iso': log_loss(X['test_labels'], probas4, eps=1e-7),
        'brier_score_probas_pos_sc': brier_score_loss(X['test_labels'], probas5),
        'log_loss_probas_pos_sc': log_loss(X['test_labels'], probas5, eps=1e-7),
        'brier_score_probas_pos_neg_sc': brier_score_loss(X['test_labels'], probas6),
        'log_loss_probas_pos_neg_sc': log_loss(X['test_labels'], probas6, eps=1e-7),
        'ece_scores': calibration_loss(X['test_labels'], expit(scores)),
        'ece_probas_pos': calibration_loss(X['test_labels'], probas1),
        'ece_probas_pos_neg': calibration_loss(X['test_labels'], probas2),
        'ece_probas_pos_neg_iso': calibration_loss(X['test_labels'], probas3),
        'ece_probas_pos_iso': calibration_loss(X['test_labels'], probas4),
        'ece_probas_pos_sc': calibration_loss(X['test_labels'], probas5),
        'ece_probas_pos_neg_sc': calibration_loss(X['test_labels'], probas6),
        'metrics_mrr': mrr_score(ranks), 
        'metrics_hits@10': hits_at_n_score(ranks, n=10),
        'metrics_mr': mr_score(ranks),
        'accuracy_per_relation': per_relation_acc,
        'accuracy_uncalib': acc_uncalib,
        'accuracy_pos': acc1,
        'accuracy_pos_neg': acc2,
        'accuracy_pos_neg_iso': acc3,
        'accuracy_pos_iso': acc4,
        'accuracy_pos_sc': acc5,
        'accuracy_pos_neg_sc': acc6
    })
        
    print(json.dumps(results[-1], indent=2))

WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
WARNING - From /home/ptabacof/.conda/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
WARNING - From /home/ptabacof/.

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "TransE",
  "loss": "self_adversarial",
  "brier_score_scores": 0.445572232212395,
  "log_loss_scores": 1.533928791554572,
  "brier_score_probas_pos": 0.14212788206615748,
  "log_loss_probas_pos": 0.4463757120985599,
  "brier_score_probas_pos_neg": 0.12403854710103454,
  "log_loss_probas_pos_neg": 0.389833326626873,
  "brier_score_probas_pos_neg_iso": 0.1240950381452728,
  "log_loss_probas_pos_neg_iso": 0.3898854834610226,
  "brier_score_probas_pos_iso": 0.14107614825076797,
  "log_loss_probas_pos_iso": 0.44215528119862046,
  "brier_score_probas_pos_sc": 0.14510261125934348,
  "log_loss_probas_pos_sc": 0.45124740933776486,
  "brier_score_probas_pos_neg_sc": 0.12403848918708212,
  "log_loss_probas_pos_neg_sc": 0.3898356655070431,
  "ece_scores": 0.4625255433962153,
  "ece_probas_pos": 0.10802915995225082,
  "ece_probas_pos_neg": 0.01067582282653529,
  "ece_probas_pos_neg_iso": 0.00699252277435059,
  "ece_probas_pos_iso": 0.08946030833209556,
  "ece_probas_pos_sc": 0.1130495

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/Am

{
  "model": "TransE",
  "loss": "pairwise",
  "brier_score_scores": 0.4999600556860562,
  "log_loss_scores": 5.233593539454379,
  "brier_score_probas_pos": 0.22494239533363558,
  "log_loss_probas_pos": 0.6373193005651496,
  "brier_score_probas_pos_neg": 0.22469787795731397,
  "log_loss_probas_pos_neg": 0.6362553057238051,
  "brier_score_probas_pos_neg_iso": 0.20299433345446838,
  "log_loss_probas_pos_neg_iso": 0.5820868916678338,
  "brier_score_probas_pos_iso": 0.20803145528953107,
  "log_loss_probas_pos_iso": 0.5936251368204077,
  "brier_score_probas_pos_sc": 0.2284992373799987,
  "log_loss_probas_pos_sc": 0.6437693785737651,
  "brier_score_probas_pos_neg_sc": 0.22469730589396622,
  "log_loss_probas_pos_neg_sc": 0.6362553495353314,
  "ece_scores": 0.4999776426222399,
  "ece_probas_pos": 0.10634056276933913,
  "ece_probas_pos_neg": 0.156289998909715,
  "ece_probas_pos_neg_iso": 0.006800646281727416,
  "ece_probas_pos_iso": 0.06314431733619326,
  "ece_probas_pos_sc": 0.1514007931267311

...

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in double_scalars
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in double_scalars
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


{
  "model": "HolE",
  "loss": "pairwise",
  "brier_score_scores": 0.23291515060139195,
  "log_loss_scores": 0.6535404976821143,
  "brier_score_probas_pos": 0.2149373772754216,
  "log_loss_probas_pos": 0.6210353871879349,
  "brier_score_probas_pos_neg": 0.21448199521229427,
  "log_loss_probas_pos_neg": 0.6206827404878698,
  "brier_score_probas_pos_neg_iso": 0.19967466235375098,
  "log_loss_probas_pos_neg_iso": 0.5834483158920643,
  "brier_score_probas_pos_iso": 0.2233954721226985,
  "log_loss_probas_pos_iso": 0.6354967023709988,
  "brier_score_probas_pos_sc": 0.21543123310995757,
  "log_loss_probas_pos_sc": 0.6216989794455341,
  "brier_score_probas_pos_neg_sc": 0.21447989833067485,
  "log_loss_probas_pos_neg_sc": 0.6206832952574213,
  "ece_scores": 0.14755065314950178,
  "ece_probas_pos": 0.09450239506696906,
  "ece_probas_pos_neg": 0.09453886579587169,
  "ece_probas_pos_neg_iso": 0.005358524883773092,
  "ece_probas_pos_iso": 0.12063815282859892,
  "ece_probas_pos_sc": 0.09445204505748

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in true_divide
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/Am

{
  "model": "HolE",
  "loss": "nll",
  "brier_score_scores": 0.33461173199179783,
  "log_loss_scores": 1.18848447930473,
  "brier_score_probas_pos": 0.24927695708889283,
  "log_loss_probas_pos": 0.6916871320512149,
  "brier_score_probas_pos_neg": 0.24927398402253484,
  "log_loss_probas_pos_neg": 0.6916804365159468,
  "brier_score_probas_pos_neg_iso": 0.2282332366747824,
  "log_loss_probas_pos_neg_iso": 0.6433693097728314,
  "brier_score_probas_pos_iso": 0.2338295060006483,
  "log_loss_probas_pos_iso": 0.657914567622181,
  "brier_score_probas_pos_sc": 0.24927900507396344,
  "log_loss_probas_pos_sc": 0.6916916541303273,
  "brier_score_probas_pos_neg_sc": 0.2492739583845122,
  "log_loss_probas_pos_neg_sc": 0.69168033400879,
  "ece_scores": 0.2687403892518376,
  "ece_probas_pos": 0.045207817395641975,
  "ece_probas_pos_neg": 0.04453298290282581,
  "ece_probas_pos_neg_iso": 0.004970803743821933,
  "ece_probas_pos_iso": 0.05295066778403932,
  "ece_probas_pos_sc": 0.04570512295254479,
  "ece

/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in double_scalars
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:201: RuntimeWarning: invalid value encountered in true_divide
  avg_pred_true = y_true[i_start:i_end].sum() / delta_count
/home/ptabacof/AmpliGraph-Lab/iclr_paper_notebooks/generate_corruptions.py:202: RuntimeWarning: invalid value encountered in double_scalars
  bin_centroid = y_prob[i_start:i_end].sum() / delta_count


In [7]:
import pandas as pd

In [8]:
def highlight_min(s):
    is_min = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_min]

In [18]:
df = pd.DataFrame(results).set_index(['model', 'loss'])
df.to_csv("main_results_fb13.csv", index=False)

In [10]:
bs = df[(c for c in df.columns if c.startswith('brier'))]
bs.columns = [c[len("brier_score_"):] for c in bs.columns]
bs.style.apply(highlight_min, axis=1)

In [11]:
ll = df[(c for c in df.columns if c.startswith('log_loss'))]
ll.columns = [c[len("log_loss_"):] for c in ll.columns]
ll.style.apply(highlight_min, axis=1)

In [12]:
print((bs.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
 .round(3)
 .to_latex()))

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   0.446 &           0.124 &               0.124 &       0.142 &           0.141 \\
1 &  DistMult &   0.473 &           0.178 &               0.170 &       0.186 &           0.192 \\
2 &   ComplEx &   0.481 &           0.177 &               0.170 &       0.182 &           0.189 \\
3 &      HolE &   0.452 &           0.229 &               0.228 &       0.242 &           0.263 \\
\bottomrule
\end{tabular}



In [13]:
print(ll.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'scores', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} &     model &  scores &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &    TransE &   1.534 &           0.390 &               0.390 &       0.446 &           0.442 \\
1 &  DistMult &   2.177 &           0.533 &               0.518 &       0.554 &           0.567 \\
2 &   ComplEx &   2.393 &           0.534 &               0.516 &       0.546 &           0.565 \\
3 &      HolE &   1.681 &           0.650 &               0.651 &       0.677 &           0.725 \\
\bottomrule
\end{tabular}



In [15]:
print(bs.reset_index()
 .query("model == 'TransE' ")
 [['loss',  'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &              loss &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &  self\_adversarial &           0.124 &               0.124 &       0.142 &           0.141 \\
1 &          pairwise &           0.225 &               0.203 &       0.225 &           0.208 \\
2 &               nll &           0.209 &               0.203 &       0.240 &           0.244 \\
3 &    multiclass\_nll &           0.146 &               0.146 &       0.162 &           0.159 \\
\bottomrule
\end{tabular}



In [16]:
print(ll.reset_index()
 .query("model == 'TransE' ")
 [['loss', 'probas_pos_neg', 'probas_pos_neg_iso', 'probas_pos', 'probas_pos_iso']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &              loss &  probas\_pos\_neg &  probas\_pos\_neg\_iso &  probas\_pos &  probas\_pos\_iso \\
\midrule
0 &  self\_adversarial &           0.390 &               0.390 &       0.446 &           0.442 \\
1 &          pairwise &           0.636 &               0.582 &       0.637 &           0.594 \\
2 &               nll &           0.614 &               0.592 &       0.676 &           0.685 \\
3 &    multiclass\_nll &           0.455 &               0.454 &       0.500 &           0.490 \\
\bottomrule
\end{tabular}



In [22]:
acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_min, axis=1)

In [23]:
print((acc*100).reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'pos_neg', 'pos_neg_iso', 'pos', 'pos_iso',  'uncalib', 'per_relation']]
 .reset_index(drop=True)
  .round(1)
 .to_latex())

\begin{tabular}{llrrrrrr}
\toprule
{} &     model &  pos\_neg &  pos\_neg\_iso &   pos &  pos\_iso &  uncalib &  per\_relation \\
\midrule
0 &    TransE &     82.4 &         82.4 &  80.7 &     80.2 &     50.0 &          82.1 \\
1 &  DistMult &     72.5 &         73.2 &  72.1 &     70.2 &     50.1 &          80.8 \\
2 &   ComplEx &     73.8 &         74.2 &  74.2 &     72.4 &     50.1 &          83.6 \\
3 &      HolE &     60.3 &         60.6 &  57.8 &     54.3 &     50.0 &          62.6 \\
\bottomrule
\end{tabular}



In [17]:
metrics = df[(c for c in df.columns if c.startswith('metrics'))]
metrics.columns = [c[len("metrics_"):] for c in metrics.columns]
metrics

hits@10            mr       mrr
model    loss                                              
TransE   self_adversarial  0.394156   3430.869359  0.295835
         pairwise          0.374099   7631.636561  0.282413
         nll               0.273269   6799.552016  0.202138
         multiclass_nll    0.402267   5050.017486  0.308702
DistMult self_adversarial  0.320103   5998.061349  0.181442
         pairwise          0.162011  11409.057599  0.095039
         nll               0.054945  16951.142734  0.028533
         multiclass_nll    0.258290   7125.716049  0.171373
ComplEx  self_adversarial  0.337273   6667.319092  0.183393
         pairwise          0.156702  12744.418257  0.092388
         nll               0.057641  21309.535899  0.025090
         multiclass_nll    0.240024   8759.282518  0.154983
HolE     self_adversarial  0.038680   8937.296949  0.017731
         pairwise          0.195024  11568.277230  0.119026
         nll               0.076097  27726.117537  0.046102
         multiclass_nll    0.274533   8594.326381  0.193907

In [21]:
print(metrics.reset_index()
 .query("loss == 'self_adversarial' ")
 [['model', 'mr', 'mrr', 'hits@10']]
 .reset_index(drop=True)
  .round(3)
 .to_latex())

\begin{tabular}{llrrr}
\toprule
{} &     model &        mr &    mrr &  hits@10 \\
\midrule
0 &    TransE &  3430.869 &  0.296 &    0.394 \\
1 &  DistMult &  5998.061 &  0.181 &    0.320 \\
2 &   ComplEx &  6667.319 &  0.183 &    0.337 \\
3 &      HolE &  8937.297 &  0.018 &    0.039 \\
\bottomrule
\end{tabular}



In [ ]:
def highlight_max(s):
    is_min = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_min]

acc = df[(c for c in df.columns if c.startswith('accuracy'))]
acc.columns = [c[len("accuracy_"):] for c in acc.columns]
acc.style.apply(highlight_max, axis=1)

In [ ]:
df.corr(method='spearman').reset_index().query("index.str.startswith('accuracy')")[['index', 'log_loss_probas_pos_neg', 'log_loss_probas_pos_neg_iso', 'log_loss_probas_pos', 'log_loss_probas_pos_iso']]

In [ ]:
df.corr(method='spearman').reset_index().query("index.str.startswith('accuracy')")[['index', 'brier_score_probas_pos_neg', 'brier_score_probas_pos_neg_iso', 'brier_score_probas_pos', 'brier_score_probas_pos_iso']]

In [ ]:
len(np.unique(X['valid'][:, 1]))

In [19]:
model = TransE(batches_count=64, seed=0, epochs=1000, k=100, eta=20,
               optimizer='adam', optimizer_params={'lr':0.0001},
               loss='self_adversarial', verbose=False)

model.fit(X['train'])

scores = model.predict(X['test'])

model.calibrate(X_valid_pos, batches_count=10, epochs=1000, positive_base_rate=0.5)
print("pos", model.calibration_parameters)
probas1 = model.predict_proba(X['test'])

model.calibrate(X_valid_pos, X_valid_neg)
print("pos neg", model.calibration_parameters)
probas2 = model.predict_proba(X['test'])

val_scores = model.predict(X['valid'])

thresholds = {r: np.median(np.sort(val_scores[X['valid'][:, 1] == r])) for r in np.unique(X['valid'][:, 1])}
thresholds_test = np.vectorize(thresholds.get)(X['test'][:, 1])
per_relation_acc = accuracy_score(X['test_labels'], scores > thresholds_test)

print(thresholds)


pos [-1.4380144, -5.5623884]
pos neg [-2.752314, -9.8970175]
{'cause_of_death': -3.5680597, 'ethnicity': -3.4997067, 'gender': -3.4051323, 'institution': -3.547462, 'nationality': -3.8507419, 'profession': -3.7040129, 'religion': -3.5918012}


In [20]:
per_relation_acc

0.8216332378223495